Part 1 : Web scraping

In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import time #required for request delays
import random
import json

from tqdm import tqdm

In [2]:
# Liste of commonly used User-Agents
USER_AGENTS = [
    # Chrome on Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    # Firefox on Linux
    'Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
    # Safari on macOS
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    # Edge on Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/91.0.864.54',
]

# Function to choose a User-Agent randomly
def get_random_headers():
    random_user_agent = random.choice(USER_AGENTS)
    return {'User-Agent': random_user_agent}


## Part 1.1 : Web scraping QS World University Rankings 2026

HTML file dowload status codes :
    200 ==> Success
    4xx ==> Client-side error based on our request
    5xx ==> Server-side error based on the website's server

In [10]:
try :
    # IMPORTANT : It is always wise to include a User-Agent to simulate a browser
    get_random_headers()
    qs_world_ranking_html = requests.get("https://www.topuniversities.com/world-university-rankings?items_per_page=150", timeout =10)
    qs_world_ranking_html.raise_for_status() # Raises an exception if the error code i 4xx or 5xx 

    soup_qs_world_ranking = BeautifulSoup(qs_world_ranking_html.text, 'html.parser')
    print(f'Downloading status of the html file 200 (OK) : ',qs_world_ranking_html.status_code)
except requests.exceptions.RequestException as e:
    print(f'Error : {e}')
    exit() # Exit if the request fails 

Note : the standard 'requests' library did not retireve the full HTML as 'uni-link' classes were missing from source code. We opted to use the JSON endpoint used to import the university URLs and metadata. 

In [11]:
def get_university_urls_from_api(api_url):
    """
    Fetch university URLs and metadata from the QS Rankings JSON endpoint.
    
    Args:
        api_url (str): The target JSON API URL.
        
    Returns:
        list: A list of dictionaries containing title, url, rank, and region.
    """
    try:
        # Execute the request using random headers to avoid bot detection
        response = requests.get(api_url, headers=get_random_headers())

        if response.status_code == 200:
            # Parse the JSON response into a Python dictionary
            data = response.json()
            
            # Extract the list of university nodes (default to empty list if not found)
            university_data = data.get("score_nodes", [])
            
            extracted_data = []
            base_url = "https://www.topuniversities.com"

            for index, uni in enumerate(university_data):
                relative_path = uni.get('path')
                
                # Reconstruct the absolute URL if the path is relative
                if relative_path and relative_path.startswith('/'):
                    full_url = base_url + relative_path
                else:
                    full_url = relative_path

                # Store metadata (crucial for later Text Mining and categorisation)
                extracted_data.append({
                    'title': uni.get('title'),
                    'url': full_url,
                    'rank': uni.get('rank'),
                    'region': uni.get('region'),
                    'country': uni.get('country')
                })

                # Visual feedback for the first 3 items of the current page
                if index < 3:
                    print(
                        f"University {index + 1}: "
                        f"{extracted_data[-1]['title']} "
                        f"(rank: {extracted_data[-1]['rank']})"
                    )

            print(f"Page processed: {len(extracted_data)} universities retrieved.")
            return extracted_data

        else:
            print(f"Request failed (Status {response.status_code}) for URL: {api_url}")
            return []

    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        return []

In [12]:
# Global list to store all universities across all pages
all_universities = []

# Loop through the desired pages (e.g., from page 0 to )
# page=0 contains ranks 1-150, page=1 contains 151-300, etc.
for page_number in range(0, 11):
    print(f"\n--- Processing Page {page_number} ---")
    
    # Construct the API URL dynamically
    target_api = f"https://www.topuniversities.com/rankings/endpoint?nid=4061771&page={page_number}&items_per_page=150&tab=indicators&region=&countries=&cities=&search=&star=&sort_by=&order_by=&program_type=&scholarship=&fee=&english_score=&academic_score=&mix_student=&loggedincache="
    
    # Call the function and extend the global list
    page_results = get_university_urls_from_api(target_api)
    all_universities.extend(page_results)

print(f"\n EXTRACTION COMPLETE: {len(all_universities)} universities ready for content scraping.")


--- Processing Page 0 ---
University 1: Massachusetts Institute of Technology (MIT) (rank: 1)
University 2: Imperial College London (rank: 2)
University 3: Stanford University (rank: 3)
Page processed: 150 universities retrieved.

--- Processing Page 1 ---
University 1: Western University (rank: 151)
University 2: University of Vienna (rank: 152)
University 3: Universiti Teknologi Malaysia  (rank: 153)
Page processed: 150 universities retrieved.

--- Processing Page 2 ---
University 1: Universidad Carlos III de Madrid (UC3M) (rank: 301)
University 2: Stellenbosch University (rank: 302)
University 3: Jagiellonian University (rank: 303)
Page processed: 150 universities retrieved.

--- Processing Page 3 ---
University 1: UNESP (rank: 451)
University 2: Johannes Gutenberg Universität Mainz (rank: 452)
University 3: Shenzhen University (rank: 453)
Page processed: 150 universities retrieved.

--- Processing Page 4 ---
University 1: Taipei Medical University (TMU) (rank: 601)
University 2: T

In [13]:
for uni in all_universities[:5]:
    print(uni['title'])
    print(uni['url'])

Massachusetts Institute of Technology (MIT)
https://www.topuniversities.com/universities/massachusetts-institute-technology-mit
Imperial College London
https://www.topuniversities.com/universities/imperial-college-london
Stanford University
https://www.topuniversities.com/universities/stanford-university
University of Oxford
https://www.topuniversities.com/universities/university-oxford
Harvard University
https://www.topuniversities.com/universities/harvard-university


In [14]:
def scrape_university_description(url):
    """
    Input: University URL
    Output: Cleaned 'About' text or None if not found
    """
    try:
        # 1. Fetch the page with random headers
        response = requests.get(url, headers=get_random_headers(), timeout=10)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        else : 
            print("Error : ", response.status_code)
            return None

        about_section =soup.find("div", class_="block about_section d-none d-md-block")

        if not about_section:
            return None

        # 3. Targeted extraction of all paragraphs inside the text container
        text_section = about_section.find("div", class_="textsection abt-overview-read")
        
        if text_section:
            paragraphs = text_section.find_all("p")
            # Join all paragraphs with double newlines
            full_text = "\n\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])
            return full_text
            
        return None

    except Exception as e:
        print(f" Error scraping {url}: {e}")
        return None

In [15]:
# Assuming 'all_universities' is your list of dictionaries from the API step
print(f"Starting the extraction of {len(all_universities)} universities...")

for index, uni in enumerate(all_universities):
    # Progress tracking
    print(f"[{index + 1}/{len(all_universities)}] Scraping: {uni['title']}")
    
    # Call the extraction function
    description = scrape_university_description(uni['url'])
    
    # Store the result back in the dictionary
    uni['QS_description'] = description
    
    # --- SAFETY PAUSE ---
    # We wait 2 seconds between each request to avoid being detected as a bot
    time.sleep(2)

print("\n✅ All Possible URLs processed!")

Starting the extraction of 1504 universities...
[1/1504] Scraping: Massachusetts Institute of Technology (MIT)
[2/1504] Scraping: Imperial College London
[3/1504] Scraping: Stanford University
[4/1504] Scraping: University of Oxford
[5/1504] Scraping: Harvard University
[6/1504] Scraping: University of Cambridge
[7/1504] Scraping: ETH Zurich
[8/1504] Scraping: National University of Singapore (NUS)
[9/1504] Scraping: UCL
[10/1504] Scraping: California Institute of Technology (Caltech)
[11/1504] Scraping: The University of Hong Kong
[12/1504] Scraping: Nanyang Technological University, Singapore (NTU Singapore)
[13/1504] Scraping: University of Chicago
[14/1504] Scraping: Peking University
[15/1504] Scraping: University of Pennsylvania
[16/1504] Scraping: Cornell University
[17/1504] Scraping: Tsinghua University
[18/1504] Scraping: University of California, Berkeley (UCB)
[19/1504] Scraping: The University of Melbourne
[20/1504] Scraping: The University of New South Wales (UNSW Sydney)

KeyboardInterrupt: 

In [ ]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_universities)

# index = False to drop the colomn of pandas index
# utf-8-sig allow to read in easily the data in a Excel file
df.to_csv("qs_university_corpus.csv", index=False, encoding='utf-8-sig')
df.to_parquet("qs_university_corpus.parquet", index=False) #

print("💾 File saved: qs_university_missions_full.csv")

💾 File saved: qs_university_missions_full.csv


We chose to save the corpus in a CSV file to facilitate visual verification of the content. However, the .parquet format will be primarily used for its compactness and processing efficiency.

## Part 1.2 : Web scraping Times Higher Education (THE)World University Ranking 2026

In [ ]:

try :
    # IMPORTANT : It is always wise to include a User-Agent to simulate a browser
    get_random_headers()
    the_world_ranking_html = requests.get("https://www.timeshighereducation.com/world-university-rankings/latest/world-ranking",headers=get_random_headers(), timeout =10)
    the_world_ranking_html.raise_for_status() # Raises an exception if the error code i 4xx or 5xx 

    soup_the_world_ranking = BeautifulSoup(the_world_ranking_html.text, 'html.parser')
    print(f'Downloading status of the html file 200 (OK) : ',the_world_ranking_html.status_code)
except requests.exceptions.RequestException as e:
    print(f'Error : {e}')
    exit() # Exit if the request fails 


Status du téléchargement du fichier html 200 (OK) :  200


In [3]:
def get_universities(file):
    try:
        # 1. Read the local HTML file
        with open(file, 'r', encoding='utf-8') as file:
            response_text = file.read()
            soup = BeautifulSoup(response_text, 'html.parser')

        # 2. JSON extraction from script tag
        script_data = soup.find('script', id='__NEXT_DATA__')
        if not script_data:
            print("Erreur : Balise __NEXT_DATA__ introuvable.")
            return None

        json_content = json.loads(script_data.string)

        # 3. Navigate throught JSOn structure to find the universities list
        table_content = json_content.get('props', {}).get('pageProps', {}).get('page', {}).get('rankingsTableConfig', {})
        ranking_data = table_content.get('rankingsData', {})
        universities_list = ranking_data.get('data', [])

        print(f"Number of universities found: {len(universities_list)}")

        # 4. Display the first university with selected keys only
        if universities_list:
            first_uni = universities_list[0]

            print("\n--- Available keys ---")
            print(list(first_uni.keys()))
            
            # Define the keys you want to see
            keys_to_show = ['rank', 'name', 'url', 'location']
            
            # Create a clean dictionary with only these keys
            filtered_view = {k: first_uni.get(k) for k in keys_to_show}
            
            print("\n--- FILTERED DATA OF THE FIRST UNIVERSITY ---")
            print(json.dumps(filtered_view, indent=4, ensure_ascii=False))
        
        return universities_list

    except Exception as e:
        print(f"An error occured: {e}")
        return None

In [ ]:
universities = get_universities("World University Rankings 2026 _ Times Higher Education (THE).html")

Number of universities found: 3118

--- FILTERED DATA OF THE FIRST UNIVERSITY ---
{
    "rank": "1",
    "name": "University of Oxford",
    "url": "/world-university-rankings/university-oxford",
    "location": "United Kingdom"
}


In [ ]:
def extract_university_data(universities):
    try:
        links = []
        base_url = "https://www.timeshighereducation.com"

        for uni in universities:
            rank = uni.get('rank')
            name = uni.get('name')
            path = uni.get('url') # Using 'url' as identified in your JSON
            location = uni.get('location')

            if name and path and rank!="Reporter":
                full_url = base_url + path
                links.append({
                    "rank": rank,
                    "name": name,
                    "location": location,
                    "url": full_url
                })

        print(f"Extracted data for {len(links)} universities.")
        return links  # This now works because it is inside a function

    except Exception as e:
        print(f"Error while accessing JSON data: {e}")
        return None

In [ ]:
url_from_universities =extract_university_data(universities=universities)

Extracted data for 2191 universities.


In [ ]:
def scrape_university_description(url):
    try :
        response = requests.get(url, headers=get_random_headers(), timeout=10)  
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        else : return None

        anchor =soup.find("div", class_="css-jxa8wh")

        if not anchor:
            print(f"Description container not found for URL: {url}")
            return None

        paragraphs = anchor.find_all("p")
        # Join all paragraphs with double newlines
        full_text = "\n\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])
        return full_text

    except Exception as e:
        print(f" Error scraping {url}: {e}")
        return None

In [ ]:
first_university_url = url_from_universities[0]['url']
print(first_university_url)
scrape_university_description(first_university_url)


https://www.timeshighereducation.com/world-university-rankings/university-oxford


'The University of Oxford is the oldest university in the English-speaking world and the world’s second oldest surviving university. While its exact founding date is unknown, there is evidence that teaching took place as far back as 1096.\n\nLocated in and around Oxford’s medieval city centre, the university comprises 44 colleges and halls, and over 100 libraries, making it the largest library system in the UK.\n\nStudents number around 22,000 in total, just over half of whom are undergraduates while over 40 per cent are international, representing 140 countries between them.\n\nCalled the "city of dreaming spires" by Victorian poet, Matthew Arnold, Oxford has the youngest population of any city in England and Wales: nearly a quarter of its residents are university students, which gives Oxford a noticeable buzz.\n\nOxford has an alumni network of over 250,000 individuals, including more than 120 Olympic medallists, 26 Nobel Prize winners, seven poets laureate, and over 30 modern world 

In [ ]:
all_universities_data = []

print(f"Starting the extraction of {len(url_from_universities)} universities...")

for index, uni in enumerate(url_from_universities):
    name = uni['name']
    rank = uni['rank']
    location = uni['location']
    url = uni['url']

    print(f"[{index + 1}/{len(url_from_universities)}] Scraping: {name}")

    description =scrape_university_description(url)

    enriched_uni = {
        "name": name,
        "rank": rank,
        "location": location,
        "description": description
    }
    all_universities_data.append(enriched_uni)
    time.sleep(2)  # Safety pause
print("--- Scraping Complete ---")

Starting the extraction of 2191 universities...
[1/2191] Scraping: University of Oxford
[2/2191] Scraping: Massachusetts Institute of Technology
[3/2191] Scraping: Princeton University
[4/2191] Scraping: University of Cambridge
[5/2191] Scraping: Harvard University
[6/2191] Scraping: Stanford University
[7/2191] Scraping: California Institute of Technology
[8/2191] Scraping: Imperial College London
[9/2191] Scraping: University of California, Berkeley
[10/2191] Scraping: Yale University
[11/2191] Scraping: ETH Zurich
[12/2191] Scraping: Tsinghua University
[13/2191] Scraping: Peking University
[14/2191] Scraping: University of Pennsylvania
[15/2191] Scraping: The University of Chicago
[16/2191] Scraping: Johns Hopkins University
[17/2191] Scraping: National University of Singapore
[18/2191] Scraping: Cornell University
[19/2191] Scraping: University of California, Los Angeles
[20/2191] Scraping: Columbia University
[21/2191] Scraping: University of Toronto
[22/2191] Scraping: UCL
[23/2

check the number of descriptions extracted 
the total number of universities with a rank = 2191
but the profile of Russian universities has been temporarily suspended
and 1 university : Delta University for Science and Technology, Shivaji University, Kolhapur and South Valley University  (Egypt) encounterd an issue while scraping

In [ ]:


#this filters out None or empty descriptions 
#it keeps only those uni where for each and every uni in the all datagrame 'description' exists and is not just whitespace

print(f"Original count: {len(all_universities_data)}")

cleaned_universities_data = [
    uni for uni in all_universities_data if uni.get('description') and uni.get('description').strip()
]

print(f"Cleaned count: {len(cleaned_universities_data)}")
print(f"Removed {len(all_universities_data) - len(cleaned_universities_data)} entries with no description.")

Original count: 2191
Cleaned count: 2106
Removed 85 entries with no description.


In [ ]:
# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(cleaned_universities_data)

# index = False to drop the colomn of pandas index
# utf-8-sig allow to read in easily the data in a Excel file
df.to_csv("the_university_corpus.csv", index=False, encoding='utf-8-sig')
df.to_parquet("the_university_corpus.parquet", index=False) #

print("File saved: the_university_corpus.csv")

💾 File saved: the_university_corpus.csv


## Part 1.3: Web scraping Times Higher Education (THE)World University Ranking 2011
It will enable to analyse the temporal evolution of key terms or themes by comparing descriptions from before 2015 (pre-SDG era)
To do so, we will need to use a Wayback Machine because the rank and the url can be extracted but the descrpition in the url are modified yearly. 

In [27]:
universities_2012 = get_universities("World University Rankings 2011-2012 _ Times Higher Education (THE).html")

Number of universities found: 402

--- Available keys ---
['rank_order', 'rank', 'name', 'scores_overall', 'scores_overall_rank', 'scores_teaching', 'scores_teaching_rank', 'scores_international_outlook', 'scores_international_outlook_rank', 'scores_industry_income', 'scores_industry_income_rank', 'scores_research', 'scores_research_rank', 'scores_citations', 'scores_citations_rank', 'iid', 'record_type', 'member_level', 'url', 'nid', 'location', 'aliases', 'closed', 'unaccredited', 'disabled', 'apply_link', 'cta_button', 'logo_url', 'enhanced']

--- FILTERED DATA OF THE FIRST UNIVERSITY ---
{
    "rank": "1",
    "name": "California Institute of Technology",
    "url": "/world-university-rankings/california-institute-technology-caltech",
    "location": "United States"
}


In [6]:
wayback_date ="20130101"
sleep_time = 2         
base_url = "https://web.archive.org/"

In [45]:
try :
    # IMPORTANT : It is always wise to include a User-Agent to simulate a browser
    get_random_headers()
    the_world_ranking_html = requests.get("https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking",headers=get_random_headers(), timeout =10)
    the_world_ranking_html.raise_for_status() # Raises an exception if the error code i 4xx or 5xx 

    soup_the_world_ranking = BeautifulSoup(the_world_ranking_html.text, 'html.parser')
    print(f'Downloading status of the html file 200 (OK) : ',the_world_ranking_html.status_code)
except requests.exceptions.RequestException as e:
    print(f'Error : {e}')
    exit() # Exit if the request fails 


In [8]:
def get_university_urls_from_2012(url):
    try :
        response = requests.get(url, headers=get_random_headers(), timeout=10)  
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        else : return None

        table =soup.find_all("table", class_="ranking main ind-OS overall")

        if not table:
            print(f"Description container not found for URL: {url}")
            return None
        else :print("Found the container")

        #print(table)

        uni_td =soup.find_all("td", class_="uni")

        #print(uni_td)

        base_url = "https://web.archive.org"
        university_links_2012 = []

        for td in uni_td:
            a_tag = td.find("a")
            if a_tag and 'href' in a_tag.attrs:
                relative_path = a_tag['href']
                full_url = base_url + relative_path
                university_links_2012.append(full_url)
        return university_links_2012


        if not uni_td:
            print(f"Description container not found for URL: {url}")
            return None
        else :print("Found the container")

    except Exception as e:
        print(f" Error scraping {url}: {e}")
        return None

In [9]:
list_of_url_2012 = get_university_urls_from_2012("https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking")
print(f"Number of university URLs extracted for 2012: {len(list_of_url_2012)}")

Found the container
Number of university URLs extracted for 2012: 200


Once the URLs have been extracted, we can begin scraping the individual university pages. However, after several requests, an archive server saturation error occurs. To mitigate this issue, we have implemented a retry loop that attempts to scrape each URL up to three times, using an exponentially increasing delay between attempts.

In [10]:
def scrape_university_description_2011(url):
    try :
        response = requests.get(url, headers=get_random_headers(), timeout=30)  
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        else : return None

        main_content =soup.find("section", class_="main-content")
        
        if not main_content:
            print(f"Description container not found for URL: {url}")
            return None
        #else :print("Found the container")

        detail = main_content.find("div", class_="details")
        #print(detail)

        
        rank_element = detail.find("span", class_="rank")
        rank = rank_element.get_text(strip=True) if rank_element else "N/A"
        #print(rank)

        header = main_content.find("header", class_="header")
        universities_2012_name = header.find("h1").get_text(strip=True)
        #print(f"University Name: {universities_2012_name}")

        info_section = main_content.find("section", class_="infomation") # Typo fixed from 'information' to 'infomation' developer mistake

        if info_section:
            paragraphs = info_section.find_all("p")
            full_text = "\n\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])
            #print(full_text)
        else :
            print(f"No information section found for URL: {url}")
            return None
        
        return {
            "rank" :rank,
            "name": universities_2012_name,
            "description": full_text
        }
            
    except Exception as e:
        print(f" Error scraping {url}: {e}")
        return None

In [87]:
scrape_university_description_2011("https://web.archive.org/web/20121007054238/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/california-institute-of-technology")

{'rank': '1',
 'name': 'California Institute of Technology',
 'description': "Caltech alumni include movie director Frank Capra, who graduated in 1918, but its 124-acre campus predates nearby Hollywood. More than 30 Caltech students have won Nobel prizes, and one alumnus - Harrison Schmitt - has walked on the Moon. Home to Nasa's Jet Propulsion Laboratory, it has a faculty of about 300 teaching around 2,000 students."}

In [88]:
scrape_university_description_2011("https://web.archive.org/web/20121008005253/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/emory-university")


{'rank': '75',
 'name': 'Emory University',
 'description': 'A "million-dollar letter" from Asa Candler, founder of the Coca-Cola Company, in 1915 helped transform a small Methodist college, founded in 1832, into today\'s research university. It has around 13,000 students and more than 3,700 academics across four undergraduate and seven graduate schools on its 600-acre Druid Hills campus.'}

In [12]:
universities_2012_data_success =[]
universities_2012_data_failed = []

retries =3

print(f"Starting the extraction of {len(list_of_url_2012)} universities...")

for url in tqdm(list_of_url_2012):
    success = False
    for attempt in range(retries):
        try :
            data = scrape_university_description_2011(url)
            if data:
                result_record = {
                    "rank": data['rank'],
                    "name": data['name'],
                    "description": data['description']
                }
                universities_2012_data_success.append(result_record)
                success = True
                break
            else : 
                print(f"Attempt {attempt + 1} failed for URL: {url}")
                time.sleep(2**attempt * 5) # Exponential backoff
        except Exception as e:
            print(f" Exception on attempt {attempt + 1} for URL: {url} - {e}")
            time.sleep(2**attempt * 5) # Exponential backoff
    if not success:
        failed_record = {
            "url": url,
            "error": "Failed after retries"
        }
        universities_2012_data_failed.append(failed_record)
    time.sleep(2)  # Safety pause
print("--- Scraping Complete ---")
print(f"Success : {len(universities_2012_data_success)}")
print(f"Failed : {len(universities_2012_data_failed)}")

Starting the extraction of 200 universities...


  6%|▋         | 13/200 [00:47<11:20,  3.64s/it]

 Error scraping https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/johns-hopkins-university: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/johns-hopkins-university (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000014885DE15D0>, 'Connection to web.archive.org timed out. (connect timeout=30)'))
Attempt 1 failed for URL: https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/johns-hopkins-university


  8%|▊         | 16/200 [01:25<23:50,  7.78s/it]

 Error scraping https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-college-london: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-college-london (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000014885E9E010>, 'Connection to web.archive.org timed out. (connect timeout=30)'))
Attempt 1 failed for URL: https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-college-london
 Error scraping https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-college-london: HTTPSConnectionPool(host='web.archive.org', p

...

 Error scraping https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-of-ottawa: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-of-ottawa (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000014885F8F150>, 'Connection to web.archive.org timed out. (connect timeout=30)'))
Attempt 1 failed for URL: https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-of-ottawa
 Error scraping https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/university-of-ottawa: HTTPSConnectionPool(host='web.archive.org', port=443): Max retrie

 92%|█████████▎| 185/200 [37:49<09:30, 38.03s/it]

 Error scraping https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/pompeu-fabra-university: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/pompeu-fabra-university (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000148860390D0>, 'Connection to web.archive.org timed out. (connect timeout=30)'))
Attempt 1 failed for URL: https://web.archive.org/web/20121006035856/http://www.timeshighereducation.co.uk/world-university-rankings/2011-12/world-ranking/institution/pompeu-fabra-university


100%|██████████| 200/200 [39:11<00:00, 11.76s/it]

--- Scraping Complete ---
Success : 197
Failed : 3


In [13]:
df_success_2012 = pd.DataFrame(universities_2012_data_success)
df_success_2012.to_csv("the_university_corpus_2012.csv", index=False, encoding='utf-8-sig')
df_success_2012.to_parquet("the_university_corpus_2012.parquet", index=False) 

print("File saved: the_university_corpus_2012.csv")

File saved: the_university_corpus_2012.csv


In [14]:
filename = "the_university_corpus_2012.parquet"

try:
    # Lecture du CSV
    df = pd.read_parquet(filename)
    print(f"File '{filename}' read. Number of universities: {len(df)}")
    
    df_filtered = df[df['rank'].astype(str) != "192"].copy()

    df_filtered.to_parquet("the_university_corpus_2012_V2.parquet", index=False)
    
except FileNotFoundError:
    print(f"File '{filename}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

File 'the_university_corpus_2012.parquet' read. Number of universities: 197


## Part 1.4: Web scraping Times Higher Education (THE)World University Ranking 2021
Waybach machine was used to scrap the 2012 website but the closest website found is 

In [4]:
universities_2021 = get_universities("2025_World University Rankings 2021 _ Times Higher Education (THE).html")

Number of universities found: 1526

--- Available keys ---
['rank_order', 'rank', 'name', 'scores_overall', 'scores_overall_rank', 'scores_teaching', 'scores_teaching_rank', 'scores_research', 'scores_research_rank', 'scores_citations', 'scores_citations_rank', 'scores_industry_income', 'scores_industry_income_rank', 'scores_international_outlook', 'scores_international_outlook_rank', 'iid', 'record_type', 'member_level', 'url', 'nid', 'location', 'stats_number_students', 'stats_student_staff_ratio', 'stats_pc_intl_students', 'stats_female_male_ratio', 'aliases', 'closed', 'unaccredited', 'disabled', 'apply_link', 'cta_button', 'logo_url', 'enhanced']

--- FILTERED DATA OF THE FIRST UNIVERSITY ---
{
    "rank": "1",
    "name": "University of Oxford",
    "url": "/world-university-rankings/university-oxford",
    "location": "United Kingdom"
}


Although 1,526 universities were found, the 2012 sample size was limited to under 200. To maintain comparative consistency, we are narrowing our scope to the Top 200 universities. We will therefore retain only the top 200 positions from the latest ranking.

In [5]:
universities_2021 = universities_2021[:200]

print(f"Number of universities extracted for 2021: {len(universities_2021)}")

Number of universities extracted for 2021: 200


In [6]:
def get_university_urls_from_2021(filename):
    try :
        with open(filename, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        soup = BeautifulSoup(html_content, 'html.parser')

        if not soup:
            print(f"Error reading HTML content from file: {filename}")
            return None
        
        data_table =soup.find("table", class_="table regWallVariants-processed wur-hash-processed wur-cols-processed wur-pagelen-processed dataTable no-footer rank-only stats usr-processed")

        if not data_table:
            print(f"Description data_table not found for file: {filename}")
            return None
        
        namesearch_td =data_table.find_all("td", class_="name namesearch")

        if not namesearch_td:
            print(f"Description container not found for file: {filename}")
            return None

        base_url = "https://web.archive.org"
        university_links_2021 = []

        for td in namesearch_td:
            a_tag = td.find("a", class_="ranking-institution-title")
            if a_tag and 'href' in a_tag.attrs:
                raw_path = a_tag['href']

                if raw_path.startswith("http"):
                    full_url = raw_path
                else:
                    full_url = base_url + raw_path
                
                clean_url = full_url.replace(" ", "").replace("mp_/", "/")  # Clean URL
                
                university_links_2021.append(clean_url)
        return university_links_2021


    except Exception as e:
        print(f" Error scraping {filename}: {e}")
        return None
        

In [7]:
list_of_url_2021 = get_university_urls_from_2021("2021_length_200_World University Rankings 2021 _ Times Higher Education (THE).html")
print(f"Number of university URLs extracted for 2021: {len(list_of_url_2021)}")
print(list_of_url_2021[:5])


Number of university URLs extracted for 2021: 200
['https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-oxford', 'https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/stanford-university', 'https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/harvard-university', 'https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/california-institute-technology', 'https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/massachusetts-institute-technology']


In [22]:
def scrape_university_description_2021(url):
    try :
        response = requests.get(url, headers=get_random_headers(), timeout=30)  
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        else : return None

        name_tag = soup.find("h1", class_="institution-info__title")
        rank_tag = soup.find("span", class_="institution-info__ranking-number")
        
        name = name_tag.get_text(strip=True) if name_tag else "Name not found"
        rank = rank_tag.get_text(strip=True) if rank_tag else "Rank not found"

        #print(name,rank)
        
        panes =soup.find_all("div", class_="pane-content")
        full_text = 'Description not found'

        paragraphs = []

        if not panes:
            print(f"Description container not found for URL: {url}")
            return None
        #else :print("Found the container")

        if panes:
            # On teste l'index 2, puis l'index 3 si le 2 est vide
            for idx in [2, 3]:
                if len(panes) > idx:
                    potential_p = panes[idx].find_all("p")
                    # On vérifie qu'il y a des <p> ET qu'ils ne sont pas vides
                    if potential_p and any(p.get_text(strip=True) for p in potential_p):
                        paragraphs = potential_p
                        break # On a trouvé, on sort de la boucle de test

        if paragraphs:
            full_text = "\n\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])
            #print(full_text)

        # 3. Retour UNIQUE du dictionnaire
        return {
            "rank": rank,
            "name": name,
            "description": full_text,
            "url": url
        }
            
    except Exception as e:
        print(f" Error scraping {url}: {e}")
        return None

In [19]:

scrape_university_description_2021("https://web.archive.org/web/20210609021249/https://www.timeshighereducation.com/world-university-rankings/university-oxford")

University of Oxford 1
The University of Oxford is the oldest university in the English-speaking world and the world’s second oldest surviving university. While its exact founding date is unknown, there is evidence that teaching took place as far back as 1096.

Located in and around Oxford’s medieval city centre, the university comprises 44 colleges and halls, and over 100 libraries, making it the largest library system in the UK.

Students number around 22,000 in total, just over half of whom are undergraduates while over 40 per cent are international, representing 140 countries between them.

Called the "city of dreaming spires" by Victorian poet, Matthew Arnold, Oxford has the youngest population of any city in England and Wales: nearly a quarter of its residents are university students, which gives Oxford a noticeable buzz.

Oxford has an alumni network of over 250,000 individuals, including more than 120 Olympic medallists, 26 Nobel Prize winners, seven poets laureate, and over 30

{'rank': '1',
 'name': 'University of Oxford',
 'description': 'The University of Oxford is the oldest university in the English-speaking world and the world’s second oldest surviving university. While its exact founding date is unknown, there is evidence that teaching took place as far back as 1096.\n\nLocated in and around Oxford’s medieval city centre, the university comprises 44 colleges and halls, and over 100 libraries, making it the largest library system in the UK.\n\nStudents number around 22,000 in total, just over half of whom are undergraduates while over 40 per cent are international, representing 140 countries between them.\n\nCalled the "city of dreaming spires" by Victorian poet, Matthew Arnold, Oxford has the youngest population of any city in England and Wales: nearly a quarter of its residents are university students, which gives Oxford a noticeable buzz.\n\nOxford has an alumni network of over 250,000 individuals, including more than 120 Olympic medallists, 26 Nobel

We have an erreur with the link used to find the description of Standford University because the number of pane-content is different.

In [20]:
scrape_university_description_2021("https://web.archive.org/web/20210527123333/https://www.timeshighereducation.com/world-university-rankings/stanford-university")

Stanford University 2
Located in the heart of Silicon Valley,Stanford Universitywas founded in 1885 by Jane and Leland Stanford, “to promote the public welfare by exercising an influence in behalf of humanity and civilization.” Since opening in 1891, Stanford's faculty and students have worked to improve the health and wellbeing of people around the world through the discovery and application of knowledge. Breakthroughs at Stanford include the first successful heart-lung transplant, the debut of the computer mouse, and the development of digital music.

Situated on 8,180 acres, Stanford is one of the largest campuses in the United States with 18 interdisciplinary research institutes and seven schools on a single campus: Graduate School of Business; School of Earth, Energy & Environmental Sciences; Graduate School of Education; School of Engineering; School of Humanities and Sciences; Law School; and School of Medicine.

Stanford has more than 16,300 students, 2,180 faculty and 1,800 po

{'rank': '2',
 'name': 'Stanford University',
 'description': "Located in the heart of Silicon Valley,Stanford Universitywas founded in 1885 by Jane and Leland Stanford, “to promote the public welfare by exercising an influence in behalf of humanity and civilization.” Since opening in 1891, Stanford's faculty and students have worked to\xa0improve the health and wellbeing\xa0of people around the world through\xa0the\xa0discovery and application of knowledge. Breakthroughs at Stanford\xa0include the first successful\xa0heart-lung transplant, the debut of the computer mouse, and the development of\xa0digital music.\n\nSituated on 8,180 acres, Stanford is one of the largest campuses in the United States with 18 interdisciplinary research institutes and seven schools on a single campus: Graduate School of Business; School of Earth, Energy & Environmental Sciences; Graduate School of Education; School of Engineering; School of Humanities and Sciences; Law School; and School of Medicine.\n\n

In [23]:
universities_2021_data_success =[]
universities_2021_data_failed = []

retries =3

print(f"Starting the extraction of {len(list_of_url_2021)} universities...")

for url in tqdm(list_of_url_2021):
    success = False
    for attempt in range(retries):
        try :
            data = scrape_university_description_2021(url)
            if data:
                result_record = {
                    "rank": data['rank'],
                    "name": data['name'],
                    "description": data['description']
                }
                universities_2021_data_success.append(result_record)
                success = True
                break
            else : 
                print(f"Attempt {attempt + 1} failed for URL: {url}")
                time.sleep(2**attempt * 5) # Exponential backoff
        except Exception as e:
            print(f" Exception on attempt {attempt + 1} for URL: {url} - {e}")
            time.sleep(2**attempt * 5) # Exponential backoff
    if not success:
        failed_record = {
            "url": url,
            "error": "Failed after retries"
        }
        universities_2021_data_failed.append(failed_record)
    time.sleep(2)  # Safety pause
print("--- Scraping Complete ---")
print(f"Success : {len(universities_2021_data_success)}")
print(f"Failed : {len(universities_2021_data_failed)}")

Starting the extraction of 200 universities...


  5%|▌         | 10/200 [00:41<13:38,  4.31s/it]

 Error scraping https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/imperial-college-london: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/imperial-college-london (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E37DA5B890>, 'Connection to web.archive.org timed out. (connect timeout=30)'))
Attempt 1 failed for URL: https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/imperial-college-london


  6%|▋         | 13/200 [01:23<28:35,  9.17s/it]

 Error scraping https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/eth-zurich: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/eth-zurich (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E37DC45550>, 'Connection to web.archive.org timed out. (connect timeout=30)'))
Attempt 1 failed for URL: https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/eth-zurich


...

 Error scraping https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-gothenburg: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-gothenburg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E37E7EE910>, 'Connection to web.archive.org timed out. (connect timeout=30)'))
Attempt 1 failed for URL: https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-gothenburg


100%|█████████▉| 199/200 [40:18<00:05,  5.10s/it]

 Error scraping https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-calgary: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-calgary (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E37E4C97D0>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))
Attempt 1 failed for URL: https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-calgary
 Error scraping https://web.archive.org/web/20210602120510/https://www.timeshighereducation.com/world-university-rankings/university-calgary: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20210602120510/https://www.timeshighereducati

100%|██████████| 200/200 [41:03<00:00, 12.32s/it]

--- Scraping Complete ---
Success : 193
Failed : 7


In [24]:
df_success_2012 = pd.DataFrame(universities_2021_data_success)
df_success_2012.to_csv("the_university_corpus_2021.csv", index=False, encoding='utf-8-sig')
df_success_2012.to_parquet("the_university_corpus_2021.parquet", index=False) 

print("File saved: the_university_corpus_2021.csv")

File saved: the_university_corpus_2021.csv


In [26]:
# rank 59 " Descripition" = 'Description not found'

filename = "the_university_corpus_2021.parquet"

try:
    df = pd.read_parquet(filename)
    print(f"File '{filename}' read. Number of universities: {len(df)}")
    
    df_filtered = df[df['rank'].astype(str) != "59"].copy()

    df_filtered.to_parquet("the_university_corpus_2012_V2.parquet", index=False)
    print(f"File 'the_university_corpus_2012_V2.parquet' read. Number of universities: {len(df_filtered)}")
    
except FileNotFoundError:
    print(f"File '{filename}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

File 'the_university_corpus_2021.parquet' read. Number of universities: 193
File 'the_university_corpus_2012_V2.parquet' read. Number of universities: 192
